In [1]:
import sqlite3
import pandas as pd


In [2]:
df = pd.read_csv('cleaned_yellow_tripdata.csv')

In [3]:
#created in-memory sqlite database
conn = sqlite3.connect(":memory:")

#loaded dataframe into sqlite table
df.to_sql(
    name="taxi_trips",
    con=conn,
    index=False,
    if_exists="replace"
)


12663116

In [4]:
#peak demand hours query
query_peak_hours = """
SELECT
    pickup_hour,
    COUNT(*) AS trip_count
FROM taxi_trips
GROUP BY pickup_hour
ORDER BY trip_count DESC
"""

peak_hours_df = pd.read_sql(query_peak_hours, conn)
peak_hours_df


,pickup_hour,trip_count
0,19,800753
1,18,794937
2,20,729948
3,21,707547
4,22,682964
5,17,664240
6,14,654689
7,15,644253
8,12,633597
9,13,631677


In [5]:
#revenue by pickup coordinates
query_revenue_zone = """
SELECT
    pickup_longitude,
    pickup_latitude,
    SUM(total_amount) AS total_revenue,
    COUNT(*) AS trip_count
FROM taxi_trips
GROUP BY pickup_longitude, pickup_latitude
ORDER BY total_revenue DESC
LIMIT 20
"""

revenue_zone_df = pd.read_sql(query_revenue_zone, conn)
revenue_zone_df


,pickup_longitude,pickup_latitude,total_revenue,trip_count
0,-74.002197,40.739552,3.950624e+06,2
1,0.000000,0.000000,3.344388e+06,227064
2,-73.948639,40.744900,1.572105e+04,1035
3,-74.186302,40.693142,9.725250e+03,726
4,-73.986717,40.722290,5.710280e+03,429
5,-73.915123,40.743576,4.462930e+03,305
6,-73.961533,40.770638,4.009300e+03,1
7,-74.003143,40.727676,3.241950e+03,232
8,-74.014336,40.711857,3.006350e+03,1
9,-73.776733,40.645378,2.967800e+03,53


In [8]:
#top revenue days
query_top_days = """
SELECT
    DATE(tpep_pickup_datetime) AS trip_date,
    SUM(total_amount) AS daily_revenue
FROM taxi_trips
GROUP BY trip_date
ORDER BY daily_revenue DESC
LIMIT 10
"""

top_days_df = pd.read_sql(query_top_days, conn)
top_days_df


,trip_date,daily_revenue
0,2015-01-18,9.936535e+06
1,2015-01-30,7.285082e+06
2,2015-01-31,7.207982e+06
3,2015-01-16,7.184200e+06
4,2015-01-23,7.134291e+06
5,2015-01-10,7.109820e+06
6,2015-01-15,6.898961e+06
7,2015-01-22,6.834479e+06
8,2015-01-29,6.775468e+06
9,2015-01-09,6.644073e+06


In [7]:
#average fare by weekday
query_avg_fare_weekday = """
SELECT
    pickup_day_of_week,
    AVG(fare_amount) AS avg_fare
FROM taxi_trips
GROUP BY pickup_day_of_week
ORDER BY pickup_day_of_week
"""

avg_fare_weekday_df = pd.read_sql(query_avg_fare_weekday, conn)
avg_fare_weekday_df


,pickup_day_of_week,avg_fare
0,0,12.026710
1,1,11.631529
2,2,11.821029
3,3,12.232657
4,4,12.067679
5,5,11.321600
6,6,11.993143


In [6]:
#monthly revenue with growth calculation
query_monthly_growth = """
WITH monthly_revenue AS (
    SELECT
        pickup_month,
        SUM(total_amount) AS revenue
    FROM taxi_trips
    GROUP BY pickup_month
)
SELECT
    pickup_month,
    revenue,
    revenue - LAG(revenue) OVER (ORDER BY pickup_month) AS revenue_change,
    ROUND(
        (revenue - LAG(revenue) OVER (ORDER BY pickup_month)) * 100.0 /
        LAG(revenue) OVER (ORDER BY pickup_month),
        2
    ) AS growth_percentage
FROM monthly_revenue
ORDER BY pickup_month
"""

monthly_growth_df = pd.read_sql(query_monthly_growth, conn)
monthly_growth_df


,pickup_month,revenue,revenue_change,growth_percentage
0,1,1.907453e+08,None,None


^Only one month shown, as the data loaded contains just one month

In [9]:
#exported sql outputs to csv
peak_hours_df.to_csv("peak_demand_hours.csv", index=False)
revenue_zone_df.to_csv("revenue_by_zone.csv", index=False)
top_days_df.to_csv("top_revenue_days.csv", index=False)
avg_fare_weekday_df.to_csv("avg_fare_by_weekday.csv", index=False)
monthly_growth_df.to_csv("monthly_growth.csv", index=False)
